In [ ]:
print("hello world")

In [1]:
import numpy as np
import matplotlib as plt

import pycuda.driver as cuda_driver
import pycuda.compiler as cuda_compiler
from pycuda.gpuarray import GPUArray

import IPythonMagic
from Timer import Timer

In [2]:
%setup_logging

Python version 3.6.6 | packaged by conda-forge | (default, Jul 26 2018, 11:48:23) [MSC v.1900 64 bit (AMD64)]


In [3]:
%cuda_context_handler context

Registering context in user workspace
Creating context
PyCUDA version 2018.1.1
CUDA version (10, 0, 0)
Driver version 10000
Using 'GeForce GTX 1050' GPU
 => compute capability: (6, 1)
 => memory: 1647 / 2048 MB available
Created context handle <1612457349264>
Using CUDA cache dir D:\orlan\Google Drive\Corso_GPU\MilanoGPU2018\Day2\cuda_cache


In [10]:
kernel_src = """
__global__ void shmemReduction(float* output,float* input,int size,int n_threads) {
    if(n_threads > size) {
        n_threads = size;
    }
        
    // First we stride through global memory and compute 
    // the maximum for every thread
    int gid = blockIdx.x*blockDim.x + threadIdx.x; // blockIdx.x is always zero because we use just one block
    
    float max_value = -99999999.99;
    for(int i = threadIdx.x; i < size; i = i + blockDim.x) {
        max_value = fmaxf(max_value, input[i]);
    }
    
    // Store the per-thread maximum in temporary
    output[threadIdx.x] = max_value;
    
    // Store the per-thread maximum in shared memory
    
    //__shared__ float max_shared[n_threads];
    //__shared__ float* max_shared = new float(n_threads);
    
    __shared__ float* max_shared;
    if(threadIdx.x == 0) {
        max_shared = new float(n_threads);
    }
    __syncthreads();
    
    max_shared[threadIdx.x] = max_value;
    
    
    // Synchronize so that all thread see the same shared memory
    __syncthreads();
    
    // Reduce up to get the global maximum
    
    /*--- __shared__ int n_tr = n_threads; *---/
    
    /*__shared__ int n_tr;
    if(threadIdx.x == 0) {
        n_tr = n_threads;
    }
    __syncthreads();
    */
    
    int n_t = n_threads;
    while(n_t > 1) {
        if((n_t % 2) == 1) {
            if(threadIdx.x < (int) n_t/2) {
               max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x],max_shared[threadIdx.x+(int)(n_t/2)+1]);
            }
            if(threadIdx.x == (int) n_t/2) {
                max_shared[(int) n_t/2] = max_shared[threadIdx.x];
            }
            n_t = (int) n_t/2 + 1;
        }
        else {
            if(threadIdx.x < n_t/2) {
               max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x],max_shared[threadIdx.x+(n_t/2)]);
            }
            n_t = n_t/2;
        }
        
        if(n_t > 32) {
            __syncthreads();
        }             
    }
    
    /*
    if(threadIdx.x < 32) {
        // Reduce from 64 to 32
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x],max_shared[threadIdx.x+32]);
    }
    //Since we have more than one active warp(threadIdx.x>32) we need to ensure that first they have finished
    __syncthreads();
    if(threadIdx.x < 16) {
        // Reduce from 32 to 16
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x],max_shared[threadIdx.x+16]);
    }
    if(threadIdx.x < 8) {
        // Reduce from 16 to 8
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x],max_shared[threadIdx.x+8]);
    }
    if(threadIdx.x < 4) {
        // Reduce from 8 to 4
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x],max_shared[threadIdx.x+4]);
    }
    if(threadIdx.x < 2) {
        // Reduce from 4 to 2
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x],max_shared[threadIdx.x+2]);
    }
    if(threadIdx.x < 1) {
        // Reduce from 2 to 1
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x],max_shared[threadIdx.x+1]);
    }
    */
    
    if(threadIdx.x == 0) {
        output[0] = max_shared[0];
    }
    
    // Delete the allocated memory to avoid leakage
    delete[] max_shared;
    
}
"""

kernel_module = cuda_compiler.SourceModule(kernel_src)
kernel_function = kernel_module.get_function("shmemReduction")

D:\Programmi (x86)\Anaconda3\envs\gpudev\lib\site-packages\ipykernel\__main__.py:109: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu
kernel.cu(10): warning: variable "gid" was declared but never referenced




In [5]:
n = 128;
a = np.random.rand(n).astype(np.float32)
print(a)

a_g = GPUArray(a.shape,dtype=np.float32)
a_g.set(a)

num_thread = 57 #Each kernel will compute the maximum between its thread local part
b = np.empty((1,num_thread)).astype(np.float32)
b_g = GPUArray(b.shape,dtype=np.float32)

[2.8650922e-01 1.2176803e-01 5.7562840e-01 8.2448352e-04 6.8618017e-01
 2.2552863e-01 8.9879340e-01 7.6742530e-01 6.0880017e-01 6.9910079e-01
 4.6182546e-01 6.3590991e-01 6.1053497e-01 1.7620215e-01 6.3402981e-01
 6.7808330e-01 6.1040664e-01 1.6976602e-01 8.3532876e-01 9.6601784e-01
 5.7241720e-01 9.6144706e-01 5.0598097e-01 5.0407541e-01 3.2029536e-01
 8.2096383e-02 6.2727332e-01 6.9745332e-01 4.5651048e-01 3.7344590e-02
 7.4160850e-01 8.2432610e-01 6.5867132e-01 3.6325777e-01 2.8772461e-01
 4.7099930e-01 8.8155526e-01 2.3367505e-01 9.6728468e-01 7.2419119e-01
 6.1870855e-01 6.7947310e-01 3.3188987e-01 6.3305962e-01 6.6442388e-01
 4.4791740e-01 4.3816480e-01 1.0202161e-01 2.5031471e-01 5.7778811e-01
 3.9351609e-02 2.1680520e-01 9.6571755e-01 8.6012983e-01 8.8545673e-02
 5.0515994e-02 3.7124911e-01 1.0575325e-01 1.6875978e-01 2.2124602e-01
 1.1053295e-01 6.6315573e-01 7.3654050e-01 4.8687893e-01 8.6275950e-02
 7.6484716e-01 3.9467329e-01 7.6191354e-01 1.0030141e-01 4.0291214e-01
 2.631

In [6]:
block_size = (num_thread,1,1)
grid_size = (1,1,1)

kernel_function(b_g,a_g,np.int32(n),np.int32(num_thread),grid=grid_size,block=block_size)

b_g.get(b)

#print(a)
print(b)
print(np.max(a))

[[0.9793563  0.16875978 0.5756284  0.59730124 0.6861802  0.9682694
  0.8987934  0.7674253  0.76484716 0.6991008  0.76191354 0.6359099
  0.61053497 0.7077698  0.6340298  0.7477974  0.61040664 0.8580928
  0.83532876 0.96601784 0.5724172  0.96144706 0.56213415 0.7329271
  0.5717207  0.9568289  0.6272733  0.6974533  0.74106234 0.74646014
  0.7914459  0.8243261  0.6586713  0.941974   0.86720043 0.4709993
  0.88155526 0.9793563  0.9672847  0.954973   0.61870855 0.6794731
  0.33188987 0.82727605 0.85594296 0.4479174  0.9684535  0.47147936
  0.6811194  0.5777881  0.19138026 0.21762682 0.96571755 0.86012983
  0.95951056 0.49669912 0.47671312]]
0.9793563
